In [2]:
#load libraries
import numpy as np
import math
from math import sqrt
from matplotlib.image import NonUniformImage
import matplotlib.pyplot as plt
%matplotlib inline
from statistics import mean
from scipy.stats import norm, lognorm
import seaborn as sns
sns.set()
sns.set_style('white')
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance, Voronoi, voronoi_plot_2d, ConvexHull, Delaunay
from collections import defaultdict
import itertools
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import sys
from itertools import combinations 
import time
from scipy import stats
from sklearn.svm import SVC
from mpl_toolkits import mplot3d
from shapely import geometry
import sobol_seq
import os,glob

#plotting style
%matplotlib inline
sns.set()
sns.set_style('white')

def boundary_search(slice_1, slice_2, num_iter=4):
    slice_2=slice_2[0][0]
    coord_1=slice_1[0:2]
    coord_2=slice_2[0:2]
    ID_1=slice_1[2]
    ID_2=slice_2[2]
    while num_iter > 0:
        midpt = (coord_1+coord_2)/2
        new_ID=model.predict(midpt.reshape(1, -1))
        
        if new_ID == ID_1:
            coord_1=midpt
        else: # new_ID==ID_2
            coord_2=midpt
            
        num_iter-=1
        
    return midpt

def point_adj_dict(data):
    coords=data[:,[0,1]]
    tri=Delaunay(coords)
    neigh=defaultdict(set)
    for p in tri.simplices:
        for i,j in itertools.combinations(p,2):
            neigh[i].add(j)
            neigh[j].add(i)
    return neigh

def grain_adj_dict(data, point_adj):
    grain_adj=defaultdict(set)
    for key in point_adj:
        grain_ID_1=data[key][2]
        for value in point_adj[key]:
            grain_ID_2=data[value][2]
            if grain_ID_1 != grain_ID_2:
                #array boundary
                data[key,3]=1
                data[value,3]=1
                grain_ID_1=int(grain_ID_1)
                grain_ID_2=int(grain_ID_2)
                #array grain
                data[key,4]=grain_ID_2
                data[value,4]=grain_ID_1
                #dict grain
                grain_adj[int(grain_ID_1)].add(grain_ID_2)
                grain_adj[int(grain_ID_2)].add(grain_ID_1)
    return grain_adj

def js_divergence_scipy(hist1,hist2):
    return (distance.jensenshannon(hist1, hist2, base=2))**2

def nearest_neighbors(values, all_values, nbr_neighbors=1):
    nn = NearestNeighbors(nbr_neighbors, metric='euclidean', algorithm='kd_tree').fit(all_values)
    dists, idxs = nn.kneighbors(values)
    return idxs

def golden(n,d=2):
    g = 1.32471795724474602596 
    alpha = np.zeros(d) 
    for j in range(d): 
        alpha[j] = pow(1/g,j+1) %1 
    z = np.zeros((n, d)) 
    seed=0.5
    for i in range(n): 
        z[i] = (seed + alpha*(i+1)) %1 
    return z

def hexagonal_sym_ops():
    #everything to calculate disorientations
    a=sqrt(0.75)

    #12 hexagonal close-packed crystal symmetry operators
    op25=np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
    op26=np.array([[-0.5, a, 0],[-a, -0.5, 0],[0, 0, 1]])
    op27=np.array([[-0.5, -a, 0],[a, -0.5, 0],[0, 0, 1]])
    op28=np.array([[0.5, a, 0],[-a, 0.5, 0],[0, 0, 1]])
    op2=np.array([[-1, 0, 0],[0, -1, 0],[0, 0, 1]])
    op30=np.array([[0.5, -a, 0],[a, 0.5, 0],[0, 0, 1]])
    op31=np.array([[-0.5, -a, 0],[-a, 0.5, 0],[0, 0, -1]])
    op32=np.array([[1, 0, 0],[0, -1, 0],[0, 0, -1]])
    op33=np.array([[-0.5, a, 0],[a, 0.5, 0],[0, 0, -1]])
    op34=np.array([[0.5, a, 0],[a, -0.5, 0],[0, 0, -1]])
    op35=np.array([[-1, 0, 0],[0, 1, 0],[0, 0, -1]])
    op36=np.array([[0.5, -a, 0],[-a, -0.5, 0],[0, 0, -1]])

    op25,op26,op27,op28,op2,op30,op31,op32,op33,op34,op35,op36

    #list of hexagonal sym ops
    return [op25,op26,op27,op28,op2,op30,op31,op32,op33,op34,op35,op36]

#input is an array in radians as follows euler=[angle_one,angle_two,angle_three]
def misorientation(euler_one, euler_two,sym_op):
    #orientation matrices
    g_one=np.array([[((math.cos(euler_one[0]))*(math.cos(euler_one[2]))-(math.sin(euler_one[0]))*(math.sin(euler_one[2]))*(math.cos(euler_one[1]))),((math.sin(euler_one[0]))*(math.cos(euler_one[2]))+(math.cos(euler_one[0]))*(math.sin(euler_one[2]))*(math.cos(euler_one[1]))),((math.sin(euler_one[2]))*(math.sin(euler_one[1])))],
                [(-(math.cos(euler_one[0]))*(math.sin(euler_one[2]))-(math.sin(euler_one[0]))*(math.cos(euler_one[2]))*(math.cos(euler_one[1]))),(-(math.sin(euler_one[0]))*(math.sin(euler_one[2]))+(math.cos(euler_one[0]))*(math.cos(euler_one[2]))*(math.cos(euler_one[1]))),((math.cos(euler_one[2]))*(math.sin(euler_one[1])))],
                [((math.sin(euler_one[0]))*(math.sin(euler_one[1]))),(-(math.cos(euler_one[0]))*(math.sin(euler_one[1]))),(math.cos(euler_one[1]))]        
                ])
   
    #euler_two
    g_two=np.array([[((math.cos(euler_two[0]))*(math.cos(euler_two[2]))-(math.sin(euler_two[0]))*(math.sin(euler_two[2]))*(math.cos(euler_two[1]))),((math.sin(euler_two[0]))*(math.cos(euler_two[2]))+(math.cos(euler_two[0]))*(math.sin(euler_two[2]))*(math.cos(euler_two[1]))),((math.sin(euler_two[2]))*(math.sin(euler_two[1])))],
                [(-(math.cos(euler_two[0]))*(math.sin(euler_two[2]))-(math.sin(euler_two[0]))*(math.cos(euler_two[2]))*(math.cos(euler_two[1]))),(-(math.sin(euler_two[0]))*(math.sin(euler_two[2]))+(math.cos(euler_two[0]))*(math.cos(euler_two[2]))*(math.cos(euler_two[1]))),((math.cos(euler_two[2]))*(math.sin(euler_two[1])))],
                [((math.sin(euler_two[0]))*(math.sin(euler_two[1]))),(-(math.cos(euler_two[0]))*(math.sin(euler_two[1]))),(math.cos(euler_two[1]))]        
                ])
    
#     #misorientation matrix
    delta_g=np.matmul(g_one,np.linalg.inv(g_two))
    
    #disorientation matrix
    delta_gd=np.matmul(sym_op,delta_g)
    
    #misorientation angle
    try:
        theta=np.arccos((np.trace(delta_gd)-1)/2)
    #error handling for rounding errors outside of domain
    except:
        if (((np.trace(delta_gd)-1)/2)>1):
            theta=0
        else:
            theta=180

    return math.degrees(theta)

#find the minimum misorientation using sym ops
def disorientation(euler_one, euler_two):
    global hex_sym_ops
    
    misorientation_list=[]
    for sym_op in hex_sym_ops:
        misorientation_list.append(misorientation(euler_one, euler_two,sym_op))
    
    #disorientation is the minimum of the 24 misorientations - for cubic symmetry 
    return min(misorientation_list)

def grain_areas(array,num_samples=200,num_bins=25,return_p=False,numerical=False):   
    global p
    global edge_grains
    
    #array to hold the areas of each grain
    grain_IDs=np.unique(array[:,5])
    areas=np.zeros((len(grain_IDs),1))
    grain_IDs_and_areas=np.concatenate((grain_IDs.reshape(-1,1),areas.reshape(-1,1)),axis=1)
    
    #create boundaries
    X=array[:,3:5]
    Y=array[:,5]
    #needs more than one grain and four points
    if (len(np.unique(Y)) < 2) or (len(X) < 4):
        return np.nan, np.nan, np.nan
    
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()

    model = SVC(kernel='rbf', C=1e10, gamma=10e-3)
    model.fit(X, Y)
    
    # create grid to evaluate model
    x = np.linspace(x_min, x_max, num_samples)
    y = np.linspace(y_min, y_max, num_samples)
    xx, yy = np.meshgrid(x, y)
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    svm_coords=np.stack((xx.ravel(),yy.ravel(),Z), axis=-1)
    
    #find area of each gridpoint
    total_area=(x_max-x_min)*(y_max-y_min)
    gridpt_area=total_area/(num_samples)**2

    #array to hold boundaries and adj grain
    bounds_grains=np.zeros((len(svm_coords),2))
    svm=np.concatenate((svm_coords,bounds_grains),axis=1)
    
    #create point ajd dict
    point_adj=point_adj_dict(svm)
    #create grain adj dict
    grain_adj=grain_adj_dict(svm, point_adj)
    
    #**************************************
    
    # for each grain
    for grain_ID_1 in grain_adj:
#         #get coords of grain boundary
#         svm_slice=svm[np.where((svm[:,2]==grain_ID_1) & (svm[:,3]==1)),0:2]
#         svm_slice=svm_slice[0]
#         try:
#             #create polygon
#             poly = geometry.Polygon(svm_slice)
#             #get area
#             poly_area=poly.area 
        #get all coords of grain
        svm_slice=svm[np.where(svm[:,2]==grain_ID_1),0:2]
        svm_slice=svm_slice[0]
        try:
            #get area
            poly_area=len(svm_slice)*gridpt_area
        except: #not enough points to create a polygon
            poly_area=0
        #append to array
        grain_IDs_and_areas[(np.where(grain_IDs_and_areas[:,0]==grain_ID_1)),1]=poly_area
            
    #double edge grains    
    row_ID=0
    for row in grain_IDs_and_areas:
        #if the grain ID is an edge
        if np.isin(row[0],edge_grains):
            #double the total grain area
            grain_IDs_and_areas[row_ID,1]*=2
        row_ID+=1      
    
    count=grain_IDs_and_areas[:,1]
    
    #**************************************
    
    count=count[np.where(count>0)]
#     return count #!!!!!
    mean=np.mean(count)
    var=np.var(count)
    if numerical==True:
        q, q_bin_edges = np.histogram(count, bins=num_bins, range=(0.01,50), density=True)
    else: #area
        q, q_bin_edges = np.histogram(count, bins=num_bins, range=(0.01,50), density=True, weights=count)
    q = np.append(q, 0)
    if return_p==True:
        return q, mean, var
    else:
        return js_divergence_scipy(p,q), mean, var

def sample(size,method):
    global file_size
    global raw_data
    global coords
    global grain_IDs
    global grain_IDs_and_edges
    global coords_and_grains
    global coords_and_grains_copy
    global edge_grains
    global hex_sym_ops
    
    if method == 'random':
        #choose random points
        raw_data_sample=raw_data[np.random.choice(raw_data.shape[0], size, replace=False), :]
    
    if method == 'square':
        phi = (np.sqrt(5)+1)/2
        ratio = np.sqrt(3)/2 # cos(60°)
        coords = raw_data[:,3:5]
        N = size
        
        N_X = int(np.sqrt(N))
        N_Y = N // N_X
        xv, yv = np.meshgrid(np.arange(N_X), np.arange(N_Y), sparse=False, indexing='xy')
        square_coords=np.concatenate((xv.reshape(-1,1), yv.reshape(-1,1)), axis=1)
        square_coords[:,0] = square_coords[:,0] * (np.amax(raw_data[:,3]) / np.amax(square_coords[:,0]))
        square_coords[:,1] = square_coords[:,1] * (np.amax(raw_data[:,4]) / np.amax(square_coords[:,1]))
        grid_sample=raw_data[nearest_neighbors(square_coords,coords), :]
        grid_sample=grid_sample[:,0]
        
        raw_data_sample=grid_sample
        
    if method == 'hex':
        phi = (np.sqrt(5)+1)/2
        ratio = np.sqrt(3)/2 # cos(60°)
        coords = raw_data[:,3:5]
        N = size
        
        N_X = int(np.sqrt(N)/ratio)
        N_Y = N // N_X
        xv, yv = np.meshgrid(np.arange(N_X), np.arange(N_Y), sparse=False, indexing='xy')
        xv = xv * ratio
        xv[::2, :] += ratio/2
        hex_coords=np.concatenate((xv.reshape(-1,1), yv.reshape(-1,1)), axis=1)
        hex_coords[:,0] *= (np.amax(raw_data[:,3]) / np.amax(hex_coords[:,0]))
        hex_coords[:,1] *= (np.amax(raw_data[:,4]) / np.amax(hex_coords[:,1]))
        grid_sample=raw_data[nearest_neighbors(hex_coords,coords), :]
        grid_sample=grid_sample[:,0]
        
        raw_data_sample=grid_sample
        
    if method == 'sobol':
        #choose 2D sobol points
        sobol = (sobol_seq.i4_sobol_generate(2, size))
        sobol[:,0]*=np.amax(raw_data[:,3])
        sobol[:,1]*=np.amax(raw_data[:,4])
        sobol_sample=raw_data[nearest_neighbors(sobol,coords), :]
        sobol_sample=sobol_sample[:,0]

        raw_data_sample=sobol_sample
        
    if method == 'gold':
        #choose 2D golden points
        golden_2D = golden(size)
        golden_2D[:,0]*=np.amax(raw_data[:,3])
        golden_2D[:,1]*=np.amax(raw_data[:,4])
        golden_sample=raw_data[nearest_neighbors(golden_2D,coords), :]
        golden_sample=golden_sample[:,0]

        raw_data_sample=golden_sample
        
    if method == 'window':
        percent=sqrt(size/file_size)
        #Full Window
        row_list=[]
        for index in range(0,len(raw_data)):
            if (raw_data[index,3]<=int((np.amax(raw_data[:,3])*percent))) and (raw_data[index,4]<=int((np.amax(raw_data[:,4])*percent))):
                row_list.append(raw_data[index,:])
            else:
                continue
        row_tuple=tuple(row_list)
        window_sample=np.vstack(row_tuple)
        
        raw_data_sample=window_sample
    
    #return JS Divergence
    return grain_areas(raw_data_sample)